In [2]:
### TO DO'S ###

#Herr/Frau zu 1/2 ändern
#phil I / phil II zu 1/2 ändern
#Frage, ob zu Alternativen Lables hinzufügen oder neue Variable draus machen

In [3]:
import pandas as pd
import numpy as np
import copy
import os

In [4]:
MasterCodebook = pd.DataFrame(
    {
        "Variable Name": ["VAR1", "VAR2", "VAR3", "VAR4", "VAR5", "VAR6", "VAR7", "VAR8", "VAR9"],
        "Label": ["DbID", "Studiengang", "Fakultät", "Geschlecht", "Jahr", "Semester", "Dozent", "Fachbereich", "Modulinfo"],
        "Alternative Labels": [["DbID"], ["Studiengang"], ["Fakultät"], ["Geschlecht"], ["Jahr"], ["Semester"], ["Dozent"], ["Fachbereich"], ["Modulinfo"]],
        "Type": ["Metainfo", "Metainfo", "Metainfo", "Metainfo", "Metainfo", "Metainfo", "Metainfo", "Metainfo", "Metainfo"],
        "Data Type": ["Numeric", "String", "String", "Numeric", "Numeric", "String", "String", "String", "String"],
        "Value Codes": ["none", "none", "1 = phil I\n 2 = phil II", "1 = männlich\n2 = weiblich", "none", "none", "none", "none", "none"],
        "Missing Code": [999, 999, 999, 999, 999, 999, 999, 999, 999]
    }
)

In [5]:
UltraData = pd.DataFrame(
 {
     "VAR1": [],
     "VAR2": [],
     "VAR3": [],
     "VAR4": [],
     "VAR5": [],
     "VAR6": [],
     "VAR7": [],
     "VAR8": [],
     "VAR9": []
 }   
)

In [6]:
#key = Variablenname aus exportierten Datensatz ("alt")
#value = Variablenname aus *MasterCodebook* ("neu")

matchingtable = {"DbID": "VAR1",
                 "Studiengang": "VAR2",
                 "SEK": "VAR3",
                 "Anrede": "VAR4",
                 "Jahr": "VAR5",
                 "Semester": "VAR6",
                 "Dozent": "VAR7",
                 "Fachbereich": "VAR8",
                 "Modulinfo": "VAR9"}

In [26]:
exportdata = pd.read_excel(io="/Users/Vale-chan/Documents/ArtologikExports/HeS20_Sek I MNGW_Robin_10068 9. Semester_Evolutionsbiologie der Organismen k.nt.fwd.1.9.xlsx", sheet_name=None, engine="openpyxl")

#WhiteSpace lösche in "Variable View" und "Data" (die beiden Datenblätter aus dem Excelexport)
for datasheet in exportdata.keys():
    for column in exportdata[datasheet]:
        exportdata[datasheet][column].dtype
        if exportdata[datasheet][column].dtype == "object":
            exportdata[datasheet][column] = exportdata[datasheet][column].str.strip()    

    Befragten ID   DbID     Angezeigter Name    Vorname      Nachname  \
0              9  10356      Lambauer, Manuel    Manuel      Lambauer   
1             11  12588        Niggli, Corsin    Corsin        Niggli   
2             19  11493  Weber (Kunz), Milena    Milena  Weber (Kunz)   
3              3  10369       Bandel, Vanessa   Vanessa        Bandel   
4              5  10389         Frey, Patrick   Patrick          Frey   
5             14  10396   Sandmeier, Fabienne  Fabienne     Sandmeier   
6             21  10405     Wettach, Michaela  Michaela       Wettach   
7             20   9931         Weder, Janine    Janine         Weder   
8              8  10373    Lambacher, Rebecca   Rebecca     Lambacher   
9             13  10387         Ruggle, Robin     Robin        Ruggle   
10            17  10367        Ventura, Fabio     Fabio       Ventura   
11            18  10385       Walser, Silvana   Silvana        Walser   
12            12  12592       Pfister, Daniel    Da

In [27]:
for _,row in exportdata["VariableView"].iterrows():

    exportlabel = row["Label"]

    found = False

    #Schaut, ob das Label schon irgendwo in den alternativen Labesl vorkommt.
    #Wenn ja, wird ein Eintrag im "matchingtabel" erstellt und der Loop bricht ab.
    for ii,xx in enumerate(MasterCodebook["Alternative Labels"]):
        if exportlabel in xx:
            matchingtable[row["Variable Name"]] = MasterCodebook["Variable Name"].iloc[ii]
            found = True
            break

    #Wenn Eintrag nicht gefunden wurde, wird das Label zu den alternativen Labels hinzugefügt.
    #Das Label wird danach neu benannt &  das alte und neue Label kommen in's "matchingtable"
    #Das ganze kommt dann als neue Zeile ins "MasterCodebook"
    if not found:
        # TODO
        # innehaue, dasses frogt, ob meh das als alternatives lable will
        z = copy.deepcopy(row)
        z["Alternative Labels"] = [row["Label"]]
        varnameneu = "VAR" + str((MasterCodebook.shape[0]+1))
        matchingtable[z["Variable Name"]] = varnameneu
        z["Variable Name"] = varnameneu
        MasterCodebook = MasterCodebook.append(z)

In [9]:
path = "/Users/Vale-chan/Documents/ArtologikExports/HeS21_Sek I EWAD_Oberholzer_1. Sem. Kurs-Nr. 10979_Klassenführung.xlsx"
exportname = os.path.basename(path)

exportname.split("_")

metainfo = {
       "Studiengang": "",
       "Jahr": "20" + exportname.split("_")[0][-2:],
       "Semester": exportname.split("_")[0][:-2],
       "Dozent": exportname.split("_")[2],
       "Fachbereich": "",       
       "Modulinfo": " ".join(exportname.split("_")[3:])[:-5]
       }


if (metainfo["Semester"] == "HeS") | (metainfo["Semester"] == "FrS"):
    metainfo["Semester"] = metainfo["Semester"][:1] + metainfo["Semester"][-1:]


studiengang_fachbereich = exportname.split("_")[1]

if studiengang_fachbereich[:3] == "Sek":
    metainfo["Studiengang"] = " ".join(studiengang_fachbereich.split(" ")[:2])
    metainfo["Fachbereich"] = " ".join(studiengang_fachbereich.split(" ")[2:])
elif studiengang_fachbereich[:3] == "KGP":
    metainfo["Studiengang"] = studiengang_fachbereich.split(" ")[0]
    metainfo["Fachbereich"] = " ".join(studiengang_fachbereich.split(" ")[1:])
elif studiengang_fachbereich[:3] == "Mas":
    metainfo["Studiengang"] = studiengang_fachbereich.split(" ")[0]
    metainfo["Fachbereich"] = " ".join(studiengang_fachbereich.split(" ")[1:])
else:
    raise ValueError(f"Studiengang nicht definiert:\nDatei: {exportname}\nStudiengang_Fachbereich: {studiengang_fachbereich}")

In [10]:
data = exportdata["Data"]

data.columns

data = data.drop(columns=[
                          "Befragten ID",
                          "Angezeigter Name ",
                          "Vorname",
                          "Nachname",
                          "Organisation",
                          "E-Mail",
                          "Adresse",
                          "Postleitzahl",
                          "Stadt",
                          "Land",
                          "Telefon",
                          "Handy",
                          "Sprache",
                          "Halbgruppe A",
                          "Halbgruppe B",
                          "Alter",
                          "Unternehmen",
                          "Beruf",
                          "Wohnsituation",
                          "Studienjahrgang",
                          "Familienstand",
                          "Studiengang",
                          "Lerngruppe",
                          "Lerngruppe klein",
                          "Funktion",
                          "Diplomtyp",
                          "Ort",
                          "Studienbereich",
                          "Organisation.1",
                          "ID"
                          ]
                        )

data

,DbID,SEK,Anrede,1.2,1.4,1.5,2.2,2.4,2.5,2.7,...,8.3,8.4,8.5,8.6,10.1,12.1,12.2,12.5,Was würden Sie an der Lehrveranstaltung ändern? Warum?,13.7 freiwillig Name:
0,10356,phil. II,Herr,2,4,4,3,4,2,4,...,4,4,3,4,4,4,3,3,Die Lehrveranstaltung dürfte noch etwas mehr d...,NaN
1,12588,phil. II,Herr,3,3,3,3,3,3,3,...,4,4,4,4,3,3,3,3,NaN,NaN
2,11493,phil. II,Frau,3,4,4,4,4,2,1,...,4,4,4,4,3,4,4,3,NaN,NaN
3,10369,phil. II,Frau,3,4,4,4,4,4,4,...,4,4,4,4,4,3,3,3,Eventuell könnten die Übungen besser an die Ve...,NaN
4,10389,phil. II,Herr,3,3,4,3,4,4,4,...,4,4,4,4,4,4,4,4,Die mitunter anspruchsvollen Aufgaben ( z . B...,Patrick Frey
5,10396,phil. II,Frau,3,4,4,4,4,3,4,...,4,4,4,4,4,3,3,3,"Derzeit ist es natürlich nicht anders möglich,...",Fabienne Sandmeier
6,10405,phil. II,Frau,3,4,4,3,4,3,4,...,4,4,4,4,4,4,3,4,NaN,NaN
7,9931,phil. II,Frau,3,4,4,4,4,4,4,...,4,4,4,4,4,3,3,3,NaN,NaN
8,10373,phil. II,Frau,3,3,3,4,4,2,4,...,4,4,3,4,3,4,3,3,"Schade, dass es jetzt im distance learning kei...",NaN
9,10387,phil. II,Herr,3,3,4,4,4,2,4,...,4,4,4,4,4,3,3,3,NaN,NaN


In [11]:
data["Studiengang"] = metainfo["Studiengang"]
data["Jahr"] = metainfo["Jahr"]
data["Semester"] = metainfo["Semester"]
data["Dozent"] = metainfo["Dozent"]
data["Fachbereich"] = metainfo["Fachbereich"]
data["Modulinfo"] = metainfo["Modulinfo"]

In [12]:
data = data.rename(columns=matchingtable)

In [13]:
###Block hier macht "data" und "UltraData" gleich gross###

columnsdata = np.asarray(data.columns)
columnsultradata = np.asarray(UltraData.columns)

#Schmeisst Kolonnen aus "data" und "UltraDAta" zusammen
union = np.union1d(columnsdata, columnsultradata)

#Kolonnen, welche nicht in "data" sind (sind in "UltraData") und daher zu "data" hinzugefügt werden müssen
notindata = np.setdiff1d(union, columnsdata)
#Kolonnen, welche nicht in "UltraData" sind (sind in "data") und daher zu "UltraData" hinzugefügt werden müssen
notinultradata = np.setdiff1d(union, columnsultradata)

for column in notindata:
    data[column] = np.NaN

for column in notinultradata:
    UltraData[column] = np.NaN

columnsdata = np.asarray(data.columns)
columnsultradata = np.asarray(UltraData.columns)

#Kontrolliert, ob die Kolonnen "data" und "UltraData" gleich sind
assert np.array_equal(np.sort(columnsdata), np.sort(columnsultradata))

In [14]:
UltraData = UltraData.append(data)

UltraData

,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,VAR8,VAR9,VAR10,...,VAR24,VAR25,VAR26,VAR27,VAR28,VAR29,VAR30,VAR31,VAR32,VAR33
0,10356.0,Sek I,phil. II,Herr,2021,HS,Oberholzer,EWAD,1. Sem. Kurs-Nr. 10979 Klassenführung,2.0,...,4.0,4.0,3.0,4.0,4.0,4.0,3.0,3.0,Die Lehrveranstaltung dürfte noch etwas mehr d...,NaN
1,12588.0,Sek I,phil. II,Herr,2021,HS,Oberholzer,EWAD,1. Sem. Kurs-Nr. 10979 Klassenführung,3.0,...,4.0,4.0,4.0,4.0,3.0,3.0,3.0,3.0,NaN,NaN
2,11493.0,Sek I,phil. II,Frau,2021,HS,Oberholzer,EWAD,1. Sem. Kurs-Nr. 10979 Klassenführung,3.0,...,4.0,4.0,4.0,4.0,3.0,4.0,4.0,3.0,NaN,NaN
3,10369.0,Sek I,phil. II,Frau,2021,HS,Oberholzer,EWAD,1. Sem. Kurs-Nr. 10979 Klassenführung,3.0,...,4.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,Eventuell könnten die Übungen besser an die Ve...,NaN
4,10389.0,Sek I,phil. II,Herr,2021,HS,Oberholzer,EWAD,1. Sem. Kurs-Nr. 10979 Klassenführung,3.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,Die mitunter anspruchsvollen Aufgaben ( z . B...,Patrick Frey
5,10396.0,Sek I,phil. II,Frau,2021,HS,Oberholzer,EWAD,1. Sem. Kurs-Nr. 10979 Klassenführung,3.0,...,4.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,"Derzeit ist es natürlich nicht anders möglich,...",Fabienne Sandmeier
6,10405.0,Sek I,phil. II,Frau,2021,HS,Oberholzer,EWAD,1. Sem. Kurs-Nr. 10979 Klassenführung,3.0,...,4.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,NaN,NaN
7,9931.0,Sek I,phil. II,Frau,2021,HS,Oberholzer,EWAD,1. Sem. Kurs-Nr. 10979 Klassenführung,3.0,...,4.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,NaN,NaN
8,10373.0,Sek I,phil. II,Frau,2021,HS,Oberholzer,EWAD,1. Sem. Kurs-Nr. 10979 Klassenführung,3.0,...,4.0,4.0,3.0,4.0,3.0,4.0,3.0,3.0,"Schade, dass es jetzt im distance learning kei...",NaN
9,10387.0,Sek I,phil. II,Herr,2021,HS,Oberholzer,EWAD,1. Sem. Kurs-Nr. 10979 Klassenführung,3.0,...,4.0,4.0,4.0,4.0,4.0,3.0,3.0,3.0,NaN,NaN
